First I pipe `fd` find, to get real filepaths but colors cause filepaths to break

In [ ]:
$Config = @{ 
    BasePath = 'H:\datasource\microsoft🧑git' 
}
push-location $BasePath

fd -tf -d2 --color=always
| select -First 6

Normally `Get-Item` can't parse paths that have control chars. 

In [ ]:
push-location $Config.BasePath

fd -tf -d2 --color=always
| select -First 6
| Get-Item

The to fix this, pipe to `Format-RemoveAnsiEscape` before `Get-Item`.

```ps1
push-location $Config.BasePath

fd -tf -d2 --color=always
| Format-RemoveAnsiEscape
| Get-Item
```
```ps1
    Directory: H:\datasource\microsoft🧑git

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           7/17/2022  9:41 AM       14322931 AdventureWorks Sales.xlsx
-a---           5/26/2022  8:39 AM            295 Wide World Importers - Sample database for SQL -
                                                  SQL Server Microsoft Docs.URL
```

You can get the longer version in [EveryoneShouldImportThese.ps1](https://github.com/ninmonkey/dotfiles_git/blob/efd8bbccc2e308e53e341808b657ade0e04420ff/powershell/EveryoneShouldImportThese.ps1#L99-L158)

In [ ]:
function Format-RemoveAnsiEscape {
    <#
    This is the abbreviated version for notebooks
    The full function is at <https://github.com/ninmonkey/dotfiles_git/blob/efd8bbccc2e308e53e341808b657ade0e04420ff/powershell/EveryoneShouldImportThese.ps1#L99-L158>
    #>
    [Alias('StripAnsi')]
    [cmdletbinding()]
    param(
        # pipe any text
        [Alias('Text')]
        [AllowNull()]
        [AllowEmptyString()]
        [parameter(Mandatory, Position = 0, ValueFromPipeline)]
        [string]$InputObject,

        # misc label
        [alias('All')]
        [switch]$StripEverything
    )
    begin {
        $Regex = @{
            StripColor = '\u001B.*?m'
            StripAll   = '\u001B.*?\p{L}'
        }
    }
    process {
        if ($null -eq $InputObject) {
            return
        }
        if ($StripEverything) {
            $InputObject -replace $Regex.StripAll, ''
        } else {
            $InputObject -replace $Regex.StripColor, ''
        }

    }
}

Now native commands using `--color=always` are able to pipe to `Get-Item` automatically!

In [ ]:
push-location $Config.BasePath

fd -tf -d2 --color=always
| Format-RemoveAnsiEscape
| Get-Item
| select -First 6
| Ft -groupBy Directory Name, LastWriteTime, Length
